In [2]:
%pylab notebook
%load_ext autoreload
%autoreload 2

#from mpl_toolkits.mplot3d import Axes3D
import scipy.constants as cst
from lammpsrun import LAMMPS, Prism
from itertools import product

from ase import Atoms, units
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write, read
from ase.neighborlist import neighbor_list
from ase.build import *
from ase.md import *
from ase.spacegroup import crystal

homepath = "D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\"
print(homepath)

Populating the interactive namespace from numpy and matplotlib
D:\Cloud Storage\GitRepositories\LAMMPS_Simulation\


Crystallographic data is obtained from https://materials.springer.com/isp/crystallographic/docs/sd_1628167

# Bulk siderite crystal

In [32]:
siderite = crystal(symbols = ['Fe', 'C', 'O'],
                    basis = [(0, 0, 0), 
                             (0, 0, 0.25),
                             (0.2748, 0, 0.25)],
                    spacegroup=167,
                    cellpar=[4.72, 4.72, 15.51, 90, 90, 120])

# Increasing size to improve QEq accuracy (>1 nm)
siderite_bulk = siderite_bulk.repeat((3, 3, 1))
siderite_bulk.center()

In [30]:
siderite_bulk.cell

array([[14.16      ,  0.        ,  0.        ],
       [-7.08      , 12.26291972,  0.        ],
       [ 0.        ,  0.        , 15.51      ]])

In [19]:
# Atoms C Fe O
siderite_bulk_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 12.011",
             "2 55.845",
             "3 15.999"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' C Fe O'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax iso 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

In [20]:
# Setting up simulation input
siderite_bulk_calc = LAMMPS(tmp_dir = homepath + "temp", 
                            parameters = siderite_bulk_reaxff_params,
                            no_data_file=True)

siderite_bulk.set_calculator(siderite_bulk_calc)
siderite_bulk_calc.atoms = siderite_bulk.copy()
siderite_bulk_calc.prism = Prism(siderite_bulk_calc.atoms.get_cell())

In [21]:
# Writing input file
siderite_bulk_calc.write_lammps_in(lammps_in = homepath + "siderite\\siderite_bulk_reaxff_minimize.in",
                                   lammps_trj = homepath + "siderite\\siderite_bulk_reaxff_minimize.lammpstrj")


In [22]:
# Running calculations
!lmp_serial -in siderite\siderite_bulk_reaxff_minimize.in -log siderite\siderite_bulk_reaxff_minimize.log

LAMMPS (16 Aug 2018)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:87)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 1 1 1
Created triclinic box = (0 0 0) to (14.16 12.2629 15.51) with tilt (-7.08 0 0)
  1 by 1 by 1 MPI processor grid
Created 1 atoms
  Time spent = 0.000473022 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 atoms
  Time spent = 0 secs
Created 1 at

In [23]:
# Reading LAMMPS output
siderite_bulk_calc.read_lammps_log(lammps_log = homepath + "siderite\\siderite_bulk_reaxff_minimize.log")
siderite_bulk_data = siderite_bulk_calc.thermo_content[-1]

siderite_bulk_calc.trajectory_out = Trajectory(homepath + "siderite\\siderite_bulk_reaxff_minimize.traj", 'w')
siderite_bulk_calc.read_lammps_trj(lammps_trj = homepath + "siderite\\siderite_bulk_reaxff_minimize.lammpstrj")
siderite_bulk_calc.trajectory_out.close()

In [24]:
siderite_bulk_traj = Trajectory(homepath + "siderite\\siderite_bulk_reaxff_minimize.traj", 'r')

siderite_bulk_traj_view = view(siderite_bulk_traj, viewer='ngl')
siderite_bulk_traj_view.view.update_spacefill(radiusType='covalent',
                                              scale=0.8,
                                              color_scheme='element',
                                              color_scale='rainbow')
siderite_bulk_traj_view.view.player.step = 1
siderite_bulk_traj_view

In [27]:
siderite_bulk_data

{'step': 243.0,
 'temp': 0.0,
 'press': -7477.5854,
 'cpu': 40.498059,
 'pxx': -10333.997,
 'pyy': -10538.272,
 'pzz': -1560.4868,
 'pxy': -0.022840917,
 'pxz': 0.0011978937,
 'pyz': 32.375931,
 'ke': 0.0,
 'pe': -34892.624,
 'etotal': -34892.624,
 'vol': 2984.2217,
 'lx': 14.652688,
 'ly': 12.6896,
 'lz': 16.04966,
 'atoms': 270.0}

# Surfaces/Slabs

Past work done with surface 104, 110, 100, 101, 001 and 012. High surface energy on 100 and 101

In [47]:
# Atoms C Fe O
siderite_slab_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 12.011",
             "2 55.845",
             "3 15.999"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' C Fe O'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax xy 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

## siderite (111)